**Notebook 01:** Qualidade dos dados e limpeza (Data Quality & Cleaning)

---



#Sistema Inteligente de Retenção de Clientes



**Objetivo**: Verificar e garantir a qualidade dos dados antes de qualquer análise ou modelagem.

**Autor**: Ivan  

**Dataset:** Telco Customer Churn (Kaggle)

**Data**: Fevereiro 2026

---

#Checklist de Qualidade



Neste notebook, vamos verificar:

1. **Estrutura dos dados** - dimensões, tipos, colunas
2. **Valores ausentes** - identificar e tratar missing values
3. **Duplicatas** - remover registros duplicados
4. **Tipos de dados** - corrigir tipos incorretos
5. **Valores inconsistentes** - padronizar categorias
6. **Outliers** - identificar valores extremos
7. **Valores ilógicos** - detectar impossibilidades
8. **Exportação** - salvar dataset limpo

#Dicionário de Dados



| Coluna | Tipo | Descrição |
|--------|------|-----------|
| customerID | String | ID único do cliente |
| gender | Categórica | 'Feminino' 'Masculino' |
| SeniorCitizen | Binária | 1=Idoso, 0=Não idoso |
| Partner | Binária | Tem parceiro? (Sim/Não) |
| Dependents | Binária | Tem dependentes? (Sim/Não) |
| tenure | Numérica | Total de Meses como cliente |
| PhoneService | Binária | Tem serviço telefônico? (Sim/Não)|
| MultipleLines | Categórica | Múltiplas linhas? (Sim/Não/Sem serviço telefonico)|
| InternetService | Categórica | DSL/Fibra/Nenhum |
| OnlineSecurity | Categórica | Segurança online? (Sim/Não/Sem serviço internet)|
| OnlineBackup | Categórica | Backup online? Segurança online? (Sim/Não/Sem serviço internet)|
| DeviceProtection | Categórica | Proteção de dispositivo? (Sim/Não/Sem serviço internet)|
| TechSupport | Categórica | Suporte técnico? (Sim/Não/Sem serviço internet)|
| StreamingTV | Categórica | TV streaming?  (Sim/Não/Sem serviço internet)|
| StreamingMovies | Categórica | Filmes streaming?  (Sim/Não/Sem serviço internet)|
| Contract | Categórica | Tipo de contrato (Mensal, Anual, Bianual)|
| PaperlessBilling | Binária | Fatura digital? (Sim/Não) |
| PaymentMethod | Categórica | Método de pagamento (Cheque eletrônico, Cheque correio, Transf. bancária (automática), Cartão de crédito (automático)|
| MonthlyCharges | Numérica | Cobrança mensal |
| TotalCharges | Numérica | Total gasto |
| Churn | Binária | Target: Cancelou? (Sim/Não) |

#1. Importação das Bibliotecas e Configuração Inicial

In [ ]:
#Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas carregadas com sucesso!")

#2. Carregamento dos Dados

**Fonte**: Dataset de Churn de Telecomunicações  
**Formato**: CSV  
**Encoding**: UTF-8

---

In [ ]:
#Carregar dados brutos
'''df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')'''
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

print("="*80)
print("DATASET CARREGADO")
print("="*80)
print(f"\n Dimensões: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
print(f"\n Tamanho em memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


#3. Inspeção Inicial

Vamos fazer uma primeira análise visual dos dados para entender sua estrutura.

In [ ]:
print("\n PRIMEIRAS 5 LINHAS:\n")
display(df.head())

In [ ]:
print("\n ÚLTIMAS 5 LINHAS:\n")
display(df.tail())

In [ ]:
print("\n AMOSTRA ALEATÓRIA:\n")
display(df.sample(5, random_state=42))

#4. Estrutura e Tipos de Dados

In [ ]:
print("\n INFORMAÇÕES DO DATASET:\n")
print(df.info())

In [ ]:
print("\n RESUMO DOS TIPOS:\n")
print(f"  • Numéricas (int/float): {df.select_dtypes(include=np.number).shape[1]}")
print(f"  • Categóricas (object): {df.select_dtypes(include='object').shape[1]}")
print(f"  • Outras: {df.select_dtypes(exclude=[np.number, 'object']).shape[1]}")

In [ ]:
print("\nESTATÍSTICAS DESCRITIVAS - NUMÉRICAS:\n")
display(df.describe())

In [ ]:
print("\nESTATÍSTICAS DESCRITIVAS - CATEGÓRICAS:\n")
display(df.describe(include='object'))

In [ ]:
print("\n VALORES ÚNICOS POR COLUNA:\n")

for col in df.columns:
    print(f"  '{col}': {df[col].nunique()} valores únicos")

In [ ]:
print("\nVALORES ÚNICOS PARA AS COLUNAS CATEGÓRICAS:\n")

categorical_cols = [
    'gender', 'SeniorCitizen', 'Partner', 'Dependents',
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn'
]

for col in categorical_cols:
    if col in df.columns:
        print(f"  '{col}': {df[col].unique().tolist()}")
    else:
        print(f"  Coluna '{col}' não encontrada no DataFrame.")

##4.1 Verificação de Tipos Incorretos


In [ ]:
print("\nVERIFICANDO COLUNAS COM TIPOS SUSPEITOS:\n")

object_cols = df.select_dtypes(include='object').columns

for col in object_cols:
    #Tentar converter para numérico
    try:
        numeric_test = pd.to_numeric(df[col], errors='coerce')

        #Se mais de 80% dos valores forem numéricos, é suspeito
        if numeric_test.notna().sum() / len(df) > 0.8:
            print(f"  '{col}' parece ser numérica mas está como object")
            print(f"     Valores únicos: {df[col].nunique()}")
            print(f"     Exemplo: {df[col].head(3).tolist()}")
    except:
        pass

#Observações Iniciais

Com base na inspeção:

1. **customerID**: Identificador único (não será usado na modelagem)
2. **Churn**: Variável alvo (Yes/No)
3. **TotalCharges**: Está como `object` mas deveria ser numérica
4. **SeniorCitizen**: Binária (0/1) mas pode ser categórica
5. Várias colunas categóricas com valores "No internet service" ou "No phone service"

**Próximos passos:**
- Converter TotalCharges para numérico
- Tratar valores ausentes
- Tratar valores duoplicados
- Analisar e Padronizar categorias (se necessário)

#5 Tratamento dos Dados

### 5.1. Investigação de 'TotalCharges'

* **Problema identificado:** TotalCharges está como `object` (texto)  

*  **Hipótese:** Pode conter espaços vazios ou caracteres não numéricos, onde estes, geralmente representam clientes novos que ainda não acumularam cobranças totais

In [ ]:
print("\nANÁLISE DA COLUNA 'TotalCharges':\n")
print(f"Tipo atual: {df['TotalCharges'].dtype}")
print(f"\nValores únicos (primeiros 10): {df['TotalCharges'].unique()[:10]}")

In [ ]:
#Verificar valores não numéricos
non_numeric = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isna()]
print(f"\nRegistros com TotalCharges não numérico: {len(non_numeric)}")

if len(non_numeric) > 0:
    print("\nExemplos:")
    display(non_numeric[['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']].head(11))


**Decisão:**

- Converter 'TotalCharges' para numérico (espaços vazios → NaN)

- Analisar relação com 'tenure'
  - Se tenure=0, preencher 'TotalCharges' com 0
  
  - Caso contrário, preencher com 'MonthlyCharges' (primeira cobrança)

In [ ]:
#Converter para numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

#Verificar nulos após conversão
nulos_total_charges = df['TotalCharges'].isna().sum()
print(f"Conversão concluída!")
print(f"Valores nulos após conversão: {nulos_total_charges}")

In [ ]:
#Analisar casos com TotalCharges nulo
if nulos_total_charges > 0:
    print("\nANÁLISE DOS CASOS COM TOTALCHARGES NULO:\n")
    nulos_df = df[df['TotalCharges'].isna()]

    print(f"\nDistribuição de tenure nos casos nulos:")
    print(nulos_df['tenure'].value_counts().sort_index())

    print(f"\nEstatísticas:")
    display(nulos_df[['tenure', 'MonthlyCharges', 'TotalCharges']].describe())

**Análise:** Todos os casos com TotalCharges nulo têm tenure = 0

**Decisão:** Preencher com 0 (cliente novo sem cobrança ainda)

In [ ]:
#Preencher valores nulos com valor = 0
df['TotalCharges'] = df['TotalCharges'].fillna(0)

print("Valores ausentes em TotalCharges tratados!")
print(f"Nulos restantes: {df['TotalCharges'].isna().sum()}")

### 5.2. Análise de Valores Ausentes

- Não foram identificados valores ausentes:

In [ ]:
missing_data = pd.DataFrame({
    'Coluna': df.columns,
    'Nulos': df.isnull().sum(),
    'Percentual (%)': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_data = missing_data[missing_data['Nulos'] > 0].sort_values('Nulos', ascending=False)

if len(missing_data) > 0:
    display(missing_data)

    # Visualização
    plt.figure(figsize=(10, 6))
    sns.barplot(data=missing_data, x='Percentual (%)', y='Coluna', palette='Reds_r')
    plt.title('Percentual de Valores Ausentes por Coluna', fontweight='bold', fontsize=14)
    plt.xlabel('Percentual (%)')
    plt.tight_layout()
    plt.show()
else:
    print("Nenhum valor ausente explícito encontrado!")

### 5.3. Verificação de Duplicatas

In [ ]:
duplicatas_totais = df.duplicated().sum()
print(f"\nLinhas completamente duplicadas: {duplicatas_totais}")

In [ ]:
#Verificar IDs únicos
ids_unicos = df['customerID'].nunique()
total_linhas = len(df)

print(f"\nIDs únicos: {ids_unicos:,}")
print(f"Total de linhas: {total_linhas:,}")

In [ ]:
if ids_unicos < total_linhas:
    ids_duplicados = total_linhas - ids_unicos
    print(f"\n{ids_duplicados} IDs duplicados encontrados!\n")

    # Mostrar exemplos
    duplicated_ids = df[df.duplicated(subset=['customerID'], keep=False)].sort_values('customerID')
    print(f"\nExemplos:")
    display(duplicated_ids.head(10))
else:
    print("\nTodos os IDs são únicos!\n")

###5.3. Padronização de Valores Categóricos

**Problemas identificados:**

- Valores como "No internet service" e "No phone service"

- Possíveis espaços em branco

- Inconsistência de maiúsculas/minúsculas

In [ ]:
object_cols = df.select_dtypes(include='object').columns.tolist()
print(f"\nTotal de colunas categóricas: {len(object_cols)}")
print(f"Colunas: {object_cols}\n")

In [ ]:
# Mostrar valores únicos das primeiras colunas
for col in object_cols[:8]:
    print(f"\n{col}: {df[col].nunique()} valores únicos")
    print(f"  Valores: {df[col].unique()}")

**Premissas**:

- **valor = 'No'**: Pode indicar que o cliente tem serviço de internet, mas escolheu não assinar o serviço específico como: 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV' ou 'StreamingMovies'

- **valor = 'No internet service'**: Pode indicar que o cliente não possui serviço de internet da empresa e consequentemente nenhum dos serviços específicos.

#### Possíveis inconsistencias entre `PhoneService` e `MultipleLines`

- Não identificado iconsistencias entre PhoneService e MultipleLines `PhoneService` = 'Yes' e `MultipleLines` = 'No phone service'.

- Não é possível padronizar na coluna MultipleLines os valores = 'No phone service' como 'No', pois é possível que o cliente tenha o serviço, porém opte em não tem MultipleLines.



In [ ]:
# Filtrar registros onde PhoneService é 'Yes' e MultipleLines é 'No phone service'
inconsistent_phone_service = df[
    (df['PhoneService'] == 'Yes') &
    (df['MultipleLines'] == 'No phone service')
]

print(f"Registros com PhoneService = 'Yes' e MultipleLines = 'No phone service': {len(inconsistent_phone_service)}")

if len(inconsistent_phone_service) > 0:
    print("\nExemplos de registros inconsistentes:")
    display(inconsistent_phone_service[['customerID', 'PhoneService', 'MultipleLines']].head())
else:
    print("Nenhuma inconsistência encontrada para PhoneService e MultipleLines.")

In [ ]:
print("\nContagem de valores para a coluna 'PhoneService':\n")
phone_service_counts = df['PhoneService'].value_counts()
display(phone_service_counts)

print(f"\nTotal de PhoneService = 'Yes': {phone_service_counts.get('Yes', 0)} clientes")
print(f"Total de PhoneService = 'No': {phone_service_counts.get('No', 0)} clientes")

In [ ]:
# Filtrar o DataFrame para clientes com PhoneService = 'Yes'
phone_service_yes_df = df[df['PhoneService'] == 'Yes']

# Contar os valores únicos na coluna MultipleLines para esse subconjunto
multiple_lines_counts = phone_service_yes_df['MultipleLines'].value_counts()

print("Distribuição de 'MultipleLines' para clientes com 'PhoneService' = 'Yes':\n")

if 'No' in multiple_lines_counts:
    print(f"  - 'MultipleLines' = 'No': {multiple_lines_counts['No']} clientes")
if 'Yes' in multiple_lines_counts:
    print(f"  - 'MultipleLines' = 'Yes': {multiple_lines_counts['Yes']} clientes")
if 'No phone service' in multiple_lines_counts:
    print(f"  - 'MultipleLines' = 'No phone service': {multiple_lines_counts['No phone service']} clientes")
else:
    print("  - 'MultipleLines' = 'No phone service': 0 clientes (não encontrado para PhoneService = 'Yes')")

#### Possíveis inconsistencias entre InternetService x serviços de contratação possíveis

- Não identificado iconsistencias

- Não é possível padronizar na coluna MultipleLines os valores = 'No internet service' e 'No', para que:

  - seja mantido a consistência de Estrutura de Dados (implifica o manuseio e a automação do processamento dessas colunas.),
  
  - Haja Clareza e Explicitação(pois apenas 'No', poderia gerar ambiguidade para quem olha a coluna isoladamente, sem verificar InternetService),
  
  - Preparação para Modelagem (pois a distinção entre 'No' e 'No internet service' permite que o modelo aprenda diferentes pesos ou padrões para cada situação).

In [ ]:
# Definir as colunas de serviço de internet a serem verificadas
internet_related_services = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Filtrar clientes que possuem InternetService
with_internet_service_customers = df[df['InternetService'] == 'Fiber optic'].copy()

# Lista para armazenar inconsistências encontradas
inconsistencies_found = []

for col in internet_related_services:
    # Filtrar registros onde InternetService é 'Yes' (especificamente 'Fiber optic' ou 'DSL')
    # e o serviço específico é 'No internet service'
    inconsistent_records = with_internet_service_customers[
        with_internet_service_customers[col] == 'No internet service'
    ]
    if not inconsistent_records.empty:
        inconsistencies_found.append((col, inconsistent_records))

print("Verificando inconsistências entre 'InternetService' = 'Yes' e serviços relacionados como 'No internet service':\n")

if not inconsistencies_found:
    print("  Nenhuma inconsistência encontrada. Todos os clientes com serviço de internet têm os serviços relacionados sem a tag 'No internet service'.")
else:
    print("  Inconsistências encontradas:")
    for col, records in inconsistencies_found:
        print(f"    - Na coluna '{col}': {len(records)} registro(s) inconsistente(s).")
        display(records[['customerID', 'InternetService', col]].head())


In [ ]:
# Definir as colunas de serviço de internet a serem verificadas
internet_service_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Filtrar clientes que não possuem InternetService
no_internet_service_customers = df[df['InternetService'] == 'No'].copy()

# Lista para armazenar inconsistências encontradas
inconsistencies_found = []

for col in internet_service_cols:
    # Filtrar registros onde InternetService é 'No', mas o serviço específico NÃO é 'No internet service'
    inconsistent_records = no_internet_service_customers[
        no_internet_service_customers[col] != 'No internet service'
    ]
    if not inconsistent_records.empty:
        inconsistencies_found.append((col, inconsistent_records))

print("Verificando inconsistências onde 'InternetService' = 'No':\n")

if not inconsistencies_found:
    print(" Nenhuma inconsistência encontrada. Todos os clientes sem serviço de internet têm os serviços relacionados como 'No internet service'.")
else:
    print("  ❌ Inconsistências encontradas:")
    for col, records in inconsistencies_found:
        print(f"    - Na coluna '{col}': {len(records)} registro(s) inconsistente(s).")
        display(records[['customerID', 'InternetService'] + internet_service_cols].head())


In [ ]:
from numpy.random import RandomState
# Definir as colunas de serviço de internet para verificar
internet_services = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Encontrar linhas onde os valores nessas colunas não são todos iguais
# Ou seja, o cliente tem uma combinação de 'Yes', 'No' ou 'No internet service'
mixed_services_rows = df[df.apply(lambda row: row[internet_services].nunique() > 1, axis=1)]

print(f"Total de clientes com serviços de internet mistos: {len(mixed_services_rows)}")

if not mixed_services_rows.empty:
    print("\nExemplos de clientes com serviços de internet mistos (as colunas 'InternetService' e os serviços listados):")
    display(mixed_services_rows[['customerID', 'InternetService'] + internet_services].head())
else:
    print("Nenhum cliente encontrado com uma mistura de 'Yes', 'No' ou 'No internet service' para os serviços de internet listados.")

In [ ]:
print("\nContagem de valores para a coluna 'InternetService':\n")
internet_service_counts = df['InternetService'].value_counts()
display(internet_service_counts)

print(f"\nTotal de InternetService = 'No': {internet_service_counts.get('No', 0)} clientes")
print(f"Total de InternetService = 'Yes' (DSL ou Fiber optic): {internet_service_counts.get('DSL', 0) + internet_service_counts.get('Fiber optic', 0)} clientes")

In [ ]:
internet_related_services = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Filtrar clientes que possuem InternetService (ou seja, não são 'No')
internet_yes_customers = df[df['InternetService'] != 'No'].copy()

# Preparar uma lista para armazenar os resultados
results = []

for col in internet_related_services:
    no_internet_service_count = (internet_yes_customers[col] == 'No internet service').sum()
    no_count = (internet_yes_customers[col] == 'No').sum()
    yes_count = (internet_yes_customers[col] == 'Yes').sum()
    results.append({
        'Coluna': col,
        'Total_Yes': yes_count,
        'Total_No': no_count,
        'No internet service': no_internet_service_count,
    })

# Criar um DataFrame a partir dos resultados
summary_df = pd.DataFrame(results)

print("\nResumo dos valores 'Yes', 'No' e 'No internet service', para clientes com InternetService = 'Yes':\n")
display(summary_df)


## 6. Detecção de Outliers

**Método:** IQR (Interquartile Range)  

**Decisão:** Identificar mas NÃO remover (outliers podem ser clientes importantes)

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
print(f"\nColunas numéricas analisadas: {numeric_cols}\n")

outliers_summary = []

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outliers_pct = (outliers / len(df)) * 100

    if outliers > 0:
        outliers_summary.append({
            'Coluna': col,
            'Outliers': outliers,
            'Percentual (%)': round(outliers_pct, 2),
            'Min': df[col].min(),
            'Max': df[col].max(),
            'Limite Inferior': round(lower_bound, 2),
            'Limite Superior': round(upper_bound, 2)
        })

if outliers_summary:
    outliers_df = pd.DataFrame(outliers_summary)
    display(outliers_df)

    # Visualização
    fig, axes = plt.subplots(1, len(numeric_cols), figsize=(5*len(numeric_cols), 5))
    if len(numeric_cols) == 1:
        axes = [axes]

    for idx, col in enumerate(numeric_cols):
        sns.boxplot(y=df[col], ax=axes[idx], color='skyblue')
        axes[idx].set_title(f'{col}', fontweight='bold')
        axes[idx].set_ylabel('')

    plt.suptitle('Boxplots - Detecção de Outliers', fontweight='bold', y=1.02, fontsize=14)
    plt.tight_layout()
    plt.show()

    print("\nDECISÃO: Outliers identificados mas MANTIDOS.")
    print("   Justificativa: Podem representar clientes VIP ou casos especiais importantes.")

else:
    print("\nNenhum outlier significativo detectado!")

## 7. Validação de Consistência

**Verificações:**

- Valores negativos em colunas que não deveriam ter

- Consistência entre tenure, MonthlyCharges e TotalCharges

### 7.1. Analisando Valores Negativos

In [ ]:
issues_found = []

# Tenure negativo
if (df['tenure'] < 0).any():
    count = (df['tenure'] < 0).sum()
    issues_found.append(f"tenure negativo: {count} casos")
    df = df[df['tenure'] >= 0]

# MonthlyCharges negativo
if (df['MonthlyCharges'] < 0).any():
    count = (df['MonthlyCharges'] < 0).sum()
    issues_found.append(f"MonthlyCharges negativo: {count} casos")
    df = df[df['MonthlyCharges'] >= 0]

# TotalCharges negativo
if (df['TotalCharges'] < 0).any():
    count = (df['TotalCharges'] < 0).sum()
    issues_found.append(f"TotalCharges negativo: {count} casos")
    df = df[df['TotalCharges'] >= 0]

if issues_found:
    print("\nPROBLEMAS ENCONTRADOS E CORRIGIDOS:")
    for issue in issues_found:
        print(f"  • {issue}")
else:
    print("\nNenhum valor ilógico encontrado!")

### 7.2. Análise de Consistência: TotalCharges vs (MonthlyCharges × tenure)

In [ ]:
# Calcular TotalCharges esperado
df['Expected_TotalCharges'] = df['MonthlyCharges'] * df['tenure']

# Calcular diferença
df['Diff'] = abs(df['TotalCharges'] - df['Expected_TotalCharges'])

# Permitir diferença de até 10% (descontos, promoções, etc.)
tolerance = df['Expected_TotalCharges'] * 0.10
inconsistent = (df['Diff'] > tolerance) & (df['tenure'] > 0)

print(f"Casos com diferença > 10%: {inconsistent.sum()} ({inconsistent.sum()/len(df)*100:.2f}%)")

In [ ]:
if inconsistent.sum() > 0:
    print("\nExemplos de inconsistências:")
    display(df[inconsistent][['tenure', 'MonthlyCharges', 'TotalCharges',
                               'Expected_TotalCharges', 'Diff']].head(10))

In [ ]:
print("\n5 Maiores Diferenças (Diff):\n")
display(df.nlargest(5, 'Diff')[['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Expected_TotalCharges', 'Diff']])

print("\n5 Menores Diferenças (Diff):\n")
display(df.nsmallest(5, 'Diff')[['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Expected_TotalCharges', 'Diff']])

In [ ]:
print("\nDECISÃO: Manter valores originais.")
print("Justificativa: Diferenças podem ser devido a descontos, promoções ou taxas únicas.")

#Remover colunas auxiliares
df = df.drop(['Expected_TotalCharges', 'Diff'], axis=1)

## 8. Resumo Final da Qualidade

In [ ]:
quality_summary = {
    'Métrica': [
        'Dimensões Finais',
        'Colunas Numéricas',
        'Colunas Categóricas',
        'Valores Nulos',
        'Duplicatas',
        'Outliers',
        'Valores Ilógicos',
        'Status'
    ],
    'Resultado': [
        f"{df.shape[0]:,} linhas × {df.shape[1]} colunas",
        df.select_dtypes(include=np.number).shape[1],
        df.select_dtypes(include='object').shape[1],
        f"{df.isnull().sum().sum()} (0.00%)",
        "0 (removidas)",
        f"{len(outliers_summary)} colunas (mantidos)",
        "0 (corrigidos)" if not issues_found else f"{len(issues_found)} (corrigidos)",
        'DATASET LIMPO E PRONTO'
    ]
}

quality_df = pd.DataFrame(quality_summary)
display(quality_df)

## 9. Exportação do Dataset Limpo

In [ ]:
df.to_csv('/content/processed_data.csv', index=False)

# Verificar
import os
print(f"Salvo em: /content/processed_data.csv")
print(f"Tamanho: {df.shape}")
print(f"Espaço: {os.path.getsize('/content/processed_data.csv') / 1024:.1f} KB")

## 10. Próximos Passos

Com o dataset limpo, seguiremos para:

**Notebook 02 - Exploratory Data Analysis:**

- Análise da distribuição de Churn
- Relação entre variáveis e churn
- Visualizações e insights de negócio
- Identificação de padrões

---

**Notebook 01 Concluído!**